<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/cognizant2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧:
Analyzing User Activity Logs
You are given a dataset of user activity logs. Each log entry contains a user_id, timestamp, and
activity_type. The dataset has duplicate entries, and some entries are missing values. You are required to:
Deduplicate the dataset.
Handle any missing values appropriately.
Determine the top 3 most frequent activity_type for each user_id.
Calculate the time spent by each user on each activity_type

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
data = [ ("U1", "2024-12-30 10:00:00", "LOGIN"), ("U1", "2024-12-30 10:05:00", "BROWSE"), ("U1", "2024-12-30 10:20:00", "LOGOUT"),
        ("U2", "2024-12-30 11:00:00", "LOGIN"), ("U2", "2024-12-30 11:15:00", "BROWSE"), ("U2", "2024-12-30 11:30:00", "LOGOUT"),
         ("U1", "2024-12-30 10:20:00", "LOGOUT"), # Duplicate entry (None, "2024-12-30 12:00:00", "LOGIN"),

# Missing user_id ("U3", None, "LOGOUT") # Missing timestamp ]
 # Define schema

schema = StructType([ StructField("user_id", StringType(), True), StructField("timestamp", StringType(), True), StructField("activity_type", StringType(), True) ])


In [20]:

from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.window import Window

from pyspark.sql.types import *

spark=SparkSession.builder.appName("learning").getOrCreate()

data=[ ("U1", "2024-12-30 10:00:00", "LOGIN"), ("U1", "2024-12-30 10:05:00", "BROWSE"), ("U1", "2024-12-30 10:20:00", "LOGOUT"),
        ("U2", "2024-12-30 11:00:00", "LOGIN"), ("U2", "2024-12-30 11:15:00", "BROWSE"), ("U2", "2024-12-30 11:30:00", "LOGOUT"),
         ("U1", "2024-12-30 10:20:00", "LOGOUT"), (None, "2024-12-30 12:00:00", "LOGIN") ]

schema=StructType([ StructField("user_id", StringType(), True), StructField("timestamp", StringType(), True), StructField("activity_type", StringType(), True) ])

df=spark.createDataFrame(data,schema).dropDuplicates()

df=df.dropna(subset=['user_id','timestamp','activity_type']).withColumn('timestamp',col('timestamp').cast(TimestampType()))

#df=df.dropna(subset=['user_id','timestamp','activity_type']).withColumn('timestamp',to_timestamp(col('timestamp')))

df.printSchema()

df.show()

window_spec=Window.partitionBy('user_id').orderBy(col('count').desc())

df1=df.groupBy('user_id','activity_type').count().withColumn('rank',row_number().over(window_spec)).filter(col('rank')<=3)


df1.show()

window_spec=Window.partitionBy('user_id').orderBy('timestamp')

df2=df.withColumn('next_timestamp',lag('timestamp').over(window_spec))

df3=df2.withColumn('time_spent',(col('next_timestamp').cast('long')-col('timestamp').cast('long')).cast('double')/60)

df3.show()

df4=df3.groupBy('user_id','activity_type').agg(sum('time_spent'))

df4.show()

root
 |-- user_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- activity_type: string (nullable = true)

+-------+-------------------+-------------+
|user_id|          timestamp|activity_type|
+-------+-------------------+-------------+
|     U1|2024-12-30 10:05:00|       BROWSE|
|     U2|2024-12-30 11:00:00|        LOGIN|
|     U1|2024-12-30 10:20:00|       LOGOUT|
|     U1|2024-12-30 10:00:00|        LOGIN|
|     U2|2024-12-30 11:15:00|       BROWSE|
|     U2|2024-12-30 11:30:00|       LOGOUT|
+-------+-------------------+-------------+

+-------+-------------+-----+----+
|user_id|activity_type|count|rank|
+-------+-------------+-----+----+
|     U1|       BROWSE|    1|   1|
|     U1|       LOGOUT|    1|   2|
|     U1|        LOGIN|    1|   3|
|     U2|       BROWSE|    1|   1|
|     U2|        LOGIN|    1|   2|
|     U2|       LOGOUT|    1|   3|
+-------+-------------+-----+----+

+-------+-------------------+-------------+-------------------+----------+